In [159]:
import numpy as np  
import matplotlib.pyplot as plt  
import pickle
from nltk.stem import WordNetLemmatizer

In [160]:
import nltk
from collections import Counter
import numpy as np

In [161]:
import os
import re
import copy
import time

In [162]:
def loadWordDictionaryPickle():
    f=open('word_Dictionary.pickle','rb')
    vocab=pickle.load(f)
    f.close()
    return vocab
vocab = loadWordDictionaryPickle()
Support_Score={}


In [163]:
def loadWordFrequencyPickle():
    f1=open('word_Frequency.pickle','rb')
    DocVocab=pickle.load(f1)
    #print(DocVocab['Facebook'])
    f1.close()
    return DocVocab
DocVocab = loadWordFrequencyPickle()


In [164]:
def uniqueWordFrequency(DocVocab):
    Word_Set_frequency={}
    wordSetSupport = {}
    tfidf_dict={}
    maxDoc=len(DocVocab.keys())
    print(maxDoc)
    for key in DocVocab.keys():
        for word in DocVocab[key]:
            if Word_Set_frequency.get(word) ==None:
                Word_Set_frequency[word]=DocVocab[key][word]
                if DocVocab[key][word]==0:
                    wordSetSupport[word]=0.0
                else:
                    wordSetSupport[word]=1.0
            else:
                if DocVocab[key][word]==0:
                    wordSetSupport[word]+=0.0
                else:
                    wordSetSupport[word]+=1.0
                Word_Set_frequency[word]+=DocVocab[key][word]
    #print(wordSetSupport['combine'])
    total1_sum=np.sum([Word_Set_frequency[key] for key in Word_Set_frequency.keys()])
    for key in wordSetSupport.keys():
            wordSetSupport[key]=wordSetSupport[key]/maxDoc
            tfidf_dict[key]=wordSetSupport[key]*(Word_Set_frequency[key]/(1.0*total1_sum))
    return Word_Set_frequency,wordSetSupport,tfidf_dict
        

In [165]:

Word_Set_frequency,wordSetSupport,tfidf_dict=uniqueWordFrequency(DocVocab)
mean1=np.mean([Word_Set_frequency[key]  for key in Word_Set_frequency.keys()])

#print([Word_Set_frequency[key]  for key in Word_Set_frequency.keys()])

#print(mean1)
#median1=np.median([Word_Set_frequency[key]  for key in Word_Set_frequency.keys()])
#print(median1)
#threshold=np.array([Word_Set_frequency[key]  for key in Word_Set_frequency.keys()])
#threshold1=np.reshape(len(threshold),1)
#kmeans = KMeans(n_clusters=4, random_state=0).fit(threshold1)
#print(kmeans)
def filterWordSetFrequencySupport(mean1,Word_Set_frequency,wordSetSupport):
    req_Dic={}
    req_Dic_Sup={}
    for key in Word_Set_frequency.keys():
        if Word_Set_frequency[key]>mean1:
            req_Dic[key]=Word_Set_frequency[key]
    for key in req_Dic.keys():
        req_Dic_Sup[key]=wordSetSupport[key]
    return req_Dic,req_Dic_Sup

req_Dic,req_Dic_Sup = filterWordSetFrequencySupport(mean1,Word_Set_frequency,wordSetSupport)

173


In [166]:
'''fp = open('TFIDF_values.pickle','rb')
tfidf_values={}
tfidf = pickle.load(fp)
#print(tfidf['Google'])
for key in tfidf.keys():
    for wordkey in tfidf[key].keys():
        if tfidf_values.get(wordkey)==None:
            tfidf_values[wordkey]=tfidf[key][wordkey]
        else:
            tfidf_values[wordkey]+=tfidf[key][wordkey]
for key in tfidf_values.keys():
    tfidf_values[key] = tfidf_values[key]/len(tfidf.keys())
#print(tfidf_values[''])
'''

"fp = open('TFIDF_values.pickle','rb')\ntfidf_values={}\ntfidf = pickle.load(fp)\n#print(tfidf['Google'])\nfor key in tfidf.keys():\n    for wordkey in tfidf[key].keys():\n        if tfidf_values.get(wordkey)==None:\n            tfidf_values[wordkey]=tfidf[key][wordkey]\n        else:\n            tfidf_values[wordkey]+=tfidf[key][wordkey]\nfor key in tfidf_values.keys():\n    tfidf_values[key] = tfidf_values[key]/len(tfidf.keys())\n#print(tfidf_values[''])\n"

In [167]:
def loadUniqueWordPickle(): 
    f1=open('unique_Wordset.pickle','rb')
    unique_word=pickle.load(f1)
    unique_word=list(unique_word)
    f1.close()
    return unique_word
unique_word = loadUniqueWordPickle()

In [168]:
def returnItemsWithMinSupport(vocab):
        maxDoc=len(vocab.keys())
        Support_Score={}
        for key in vocab.keys():
            for words in vocab[key]:
                if Support_Score.get(words) is None:
                    Support_Score[words]=1
                else:
                    Support_Score[words]+=1
        for key in Support_Score.keys():
            Support_Score[key]=Support_Score[key]/maxDoc
        return Support_Score

In [169]:
Support_Score=returnItemsWithMinSupport(vocab)

In [170]:
def getData():
    words=[]
    with open("data1.txt","r") as f:
        x  = f.readlines()
        for word in x:
            words.append(word.replace("\n",""))
            
    return words


In [171]:
def FilterDictionary(A,Support_Score):
    filteredDict=[]
    SScore={}
    for word in A:
        if word in unique_word:
            filteredDict.append(word)
            SScore[word]=Support_Score[word]
    return filteredDict,SScore

In [172]:
def LemmatizeWord(A):
    WN=WordNetLemmatizer()
    i=0;
    maxLen=len(A)
    while i<maxLen:
        A[i]=WN.lemmatize(A[i])
        i+=1
    return A
        

In [173]:
def ListToDic(vocab):
    my_dict={}
    for doc in vocab.keys():
        my_dict = {k: 0 for k in vocab[doc] }
        vocab[doc]=my_dict
    
    return vocab

In [174]:
vocab=ListToDic(vocab)

In [175]:
def confidence(A,B):
    ScoreMatrix=np.zeros((len(A),len(B)))
    print(len(B))
    print(len(A))
    for index1,word1 in enumerate(A):# A is list of words
        for index2,word2 in enumerate(B):
            common=0
            totA=0
            for doc in vocab.keys():
                if  vocab[doc].get(word1) is not None :
                    totA+=1
                    if vocab[doc].get(word2) is not None:
                        common+=1
                    
            if totA==0:
                print(word1,word2)
            ScoreMatrix[index1][index2]=common/totA
    return ScoreMatrix

In [176]:
A=getData()
requiredDicti ={}

#print(len(A))
A,Ascore=FilterDictionary(A,Support_Score)
A=LemmatizeWord(A)
for key in req_Dic.keys():
    if key not in A:
        requiredDicti[key]=req_Dic[key]
B=[]
#print(requiredDicti['software'])
for item,value in requiredDicti.items():
    B.append(item)
#print('software' in B )
#print(len(B))
#print(B)
confidenceMatrix=confidence(A,B)
#print(Ascore)

1044
68


my_dict

In [177]:
min_support=0.0
min_confidence=0.99


In [178]:
def SelectAssociationRules(confidenceMatrix,min_support,min_confidence):
    row,col=confidenceMatrix.shape
    print(row,col)
    required={}
    for i in range(0,row):
        for j in range(0,col):
            word=A[i]
            score=Support_Score[word]
            word2=B[j]
            if confidenceMatrix[i][j]>=min_confidence and score>min_support:
                if required.get(word) ==None and (word!='' and word2!='') :
                    required[word]=[word2]
                else:
                    if (word!='' and word2!=''):
                        required[word].append(word2)
                #print(word,'-->',word2)
                #print("\n")
    return required

In [179]:
rules=SelectAssociationRules(confidenceMatrix,min_support,min_confidence)

68 1044


In [180]:
'''def MeanOFMean(confidenceMatrix):
    mean1=[]
    row,col=confidenceMatrix.shape
    for i in range(0,row):
            mean1.append(np.median(confidenceMatrix[i]))
#             print(mean1)
    return np.mean(mean1)
MeanOFMean(confidenceMatrix)
'''

'def MeanOFMean(confidenceMatrix):\n    mean1=[]\n    row,col=confidenceMatrix.shape\n    for i in range(0,row):\n            mean1.append(np.median(confidenceMatrix[i]))\n#             print(mean1)\n    return np.mean(mean1)\nMeanOFMean(confidenceMatrix)\n'

In [181]:
def Find(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', string) 
    return url 

In [182]:
def printSummary(rule_Dict,outputfile):
    list_of_lines_to_print = []
    print(rule_Dict.keys())
    
    #print(rule_Dict['personal_information']['DuckDuckGo does not collect or share personal information'])
    
    #rule_Dict[key][key2] for key2 in rule_Dict[key]
    with open(outputfile,"w") as file1:
        for key in rule_Dict.keys():
            testList=rule_Dict[key]
            print([ele[0] for ele in testList])
            mean8 = np.mean([ele[0][1] for ele in testList])
            print(mean8)
            for rows in testList:
                #print(rows[0])
                if rows[0][1]>=mean8:
                    file1.write(rows[0][0]+"\n\n")
               
                    #list_of_lines_to_print.append(key2)
                    
            

In [183]:

def doSummarization(filename,filepath):
    outputfile = filepath+'summary/'+filename[:-4]+'Summary'+'.txt'
    
    filename = filepath+filename
    linesData =[]
    rule_Dict={}
    with open(filename,encoding="utf8") as file:
        data = file.readlines()
        #print(data)
        
        for line2 in data:
            linedata=line2.split(".")
            for line in linedata:
                s = Find(line)
                #print("url:", s)
                if len(s)<=0:
                    
                    line = line.replace("\n","")
                    line = line.replace("-","")
                    line1=line.lower()
                    #print("line:",line)
                    tokens=nltk.word_tokenize(line1)
                    if len(tokens)>1:
                        tokens=nltk.word_tokenize(line)
                        #print(tokens)
                        flag=0
                        weight=0
                        line_dict=[]
                        for token in tokens:
                            if rules.get(token)!=None:
                                for word2 in rules[token]:
                                    if word2 in tokens:
                                        flag=1
                                        #print("Printing this line beacuse of ",token,"-->",word2)
                                        #print('--> ',line,".\n")
                                        linesData.append(line)
                                        for token1 in tokens:
                                            if tfidf_dict.get(token1) is not None:
                                                weight=weight+tfidf_dict[token1]
                                        line_dict.append([line,weight])
                                        break
                            if flag==1:
                                break
                        
                        if flag==1:
                                w1=token.lower()
                                w2=word2.lower()
                                str1=w1+'_'+w2
                                if rule_Dict.get(str1) is None:
                                    rule_Dict[str1]=[line_dict]
                                else:
                                    rule_Dict[str1].append(line_dict)
    printSummary(rule_Dict,outputfile)

test_path = "Datasets/testdata/"
test_files = os.listdir(test_path)  
print(test_files)
#doSummarization("Datasets/testdata/Github.txt")
for file in test_files:
    if os.path.isdir(test_path+file):
        pass
    else:
        print("doing for ",file)
        doSummarization(file,test_path)

['DuckDuckGo.txt', 'Github.txt', 'Microsoft.txt', 'summary', 'Youtube.txt']
doing for  DuckDuckGo.txt
dict_keys(['personal_information', 'cookie_information', 'cookie_use'])
[['DuckDuckGo does not collect or share personal information', 0.023599395076301835], [' We call this sharing of personal information "search leakage', 0.02006900662399695], [' Search engines could lose data, or get hacked, or accidentally expose data due to security holes or incompetence, all of which has happened with personal information on the Internet', 0.032093380208916035], [' However, search engines are not legally obligated to collect personal information in the first place', 0.022546690018240105], [' And this information (your search history) can be pretty personal', 0.019881161302634804], ['For these reasons, DuckDuckGo takes the approach to not collect any personal information', 0.021628547610480695], [" However, in our case, we don't expect any because there is nothing useful to give them since we don'

In [274]:
main():
    

True


[1, 2]
